<a href="https://colab.research.google.com/github/costajr2019/Desafio_LLM/blob/main/desafio_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio da Imersão IA

[Imersão Inteligência Artificial 2ª - Desafio da Aula 04](https://cursos.alura.com.br/imersoes/aulas/aula-4-criando-seu-proprio-chatbot-com-a-gemini-api-no-google-colab-c131)

## Descrição Aprimorada dos Objetivos deste Projeto Colab

### Objetivo Geral

Desenvolver um script Python que utilize o LLM (Large Language Model) da Google para realizar análises e fornecer feedback detalhado sobre códigos Python escritos por alunos.

### Funcionalidades Específicas

* **Análise de Código:** O script deverá analisar códigos Python (postados por alunos) localizados em uma pasta chamada "respostas". Cada código estará em um arquivo individual.
* **Comparação com Gabarito:** O código do aluno será comparado com um código gabarito presente em um arquivo chamado "gabarito.txt".
* **Feedback Automatizado:** Através do LLM da Google, o script irá gerar feedback detalhado sobre o código do aluno, identificando erros, sugerindo melhorias e fornecendo orientações para otimização.
* **Armazenamento de Feedback:** O feedback gerado pelo LLM será armazenado nos respectivos arquivos dos alunos na pasta "feedback". O feedback será inserido como comentários no código, facilitando a consulta e compreensão por parte dos alunos.

### Benefícios do Projeto

* **Aprimoramento da Aprendizagem:** O feedback automatizado e personalizado auxiliará os alunos na identificação de erros e na compreensão de conceitos importantes em Python, promovendo um aprendizado mais eficaz e direcionado.
* **Eficiência na Correção:** A automatização da análise e do feedback otimiza o tempo de trabalho dos professores, liberando-os para se concentrarem em tarefas mais complexas e na interação individualizada com os alunos.
* **Escalabilidade:** O script pode ser facilmente adaptado para analisar um grande número de códigos, tornando-o ideal para turmas com muitos alunos ou para cursos online com alto volume de inscrições.

### Aplicações

* **Cursos de Programação:** O projeto pode ser utilizado em cursos de programação para auxiliar na avaliação de tarefas e na entrega de feedback aos alunos.
* **Treinamentos Online:** O script pode ser integrado a plataformas de treinamento online para fornecer feedback em tempo real aos participantes durante a realização de exercícios práticos.
* **Competições de Programação:** O projeto pode ser utilizado como ferramenta de avaliação em competições de programação, automatizando a análise e a classificação dos códigos submetidos pelos participantes.

### Observações

* O script Python deverá ser implementado de forma clara, concisa e bem documentada, facilitando sua compreensão e utilização por outros usuários.
* O LLM da Google pode ser acessado através da API oficial do Google AI.
* O script poderá ser adaptado para analisar outros tipos de arquivos, como arquivos de texto com outros formatos de linguagem de programação.

### Conclusão

Este projeto Colab apresenta um grande potencial para aprimorar o processo de aprendizado e avaliação em cursos de programação, otimizando o tempo de trabalho dos professores e promovendo uma experiência de aprendizado mais personalizada e eficaz para os alunos.

##Dados

| Arquivo | Conteúdo | Local de Armazenamento |
| ---| --- | --- |
| gabarito.txt | Script que produz a solução para o problema proposto.| Pasta Raíz |
| prompt.txt | Prompt que descreve o conteúdo e a forma que o LLM deve apresentar sua análise. | Pasta Raíz
| resposta_alunoN.txt | Script do N-ésimo aluno a ser avaliado pelo LLM | Pasta "respostas"

##Saída
| Arquivo | Conteúdo | Local de Armazenamento |
|---|---| --- |
| feedback_resposta_alunoN.txt | Feedback do LLM sobre o Script do N-ésimo aluno | Pasta "feedback"

##Metodologia
1. Para cada aluno, concatenar os conteúdos dos arquivos "prompt.txt", "gabarito.txt", e "resposta_alunoN.txt" para formar uma string contendo o prompt completo.
2. Obter da LLM a avaliação do script Python de cada aluno a partir do respectivo prompt completo.
3. Gravar as avaliações em arquivos separados na pasta "feedback"


A função `concatenar_strings` tem como objetivo concatenar o conteúdo de três strings em uma única string formatada. Essa string final conterá o prompt do exercício, a resposta do aluno, e o gabarito, separados por marcadores visuais.

### Argumentos

* `conteudo_prompt`: Uma string contendo o prompt do exercício.
* `conteudo_resposta`: Uma string contendo a resposta do aluno ao exercício.
* `conteudo_gabarito`: Uma string contendo o gabarito do exercício.

### Retorno

A função retorna uma única string contendo a concatenação formatada do prompt, resposta e gabarito.


In [1]:
def concatenar_strings(conteudo_prompt, conteudo_resposta, conteudo_gabarito):
  """
  Função para concatenar o conteúdo de três strings em uma única string.

  """

  # Cria a string separadora
  separador = "-" * 50 + "\n"

  # Concatena os conteúdos dos arquivos
  texto_concatenado = (
      conteudo_prompt
      + 2 * "\n"
      + "**RESPOSTA DO ALUNO**"
      + "\n"
      + separador
      + conteudo_resposta
      + "\n"
      + "\n"
      + "**GABARITO**"
      + "\n"
      + separador
      + conteudo_gabarito
  )

  return texto_concatenado

A função `processar_respostas_alunos` tem como objetivo processar todas as respostas dos alunos em uma pasta especificada. Essa função lê os arquivos de prompt, resposta do aluno e gabarito, concatena seus conteúdos e armazena o resultado em um dicionário.

### Argumentos

* `pasta_respostas`: O caminho para a pasta que contém as respostas dos alunos.

### Retorno

A função retorna um dicionário onde as chaves são os nomes dos arquivos de resposta (sem a extensão `.txt`) e os valores são as strings concatenadas contendo as respostas, prompts e gabaritos.

### Detalhes da Implementação

**1. Importação do Módulo `os`:**

A função inicia importando o módulo `os` da biblioteca padrão do Python. O módulo `os` fornece funções para trabalhar com o sistema operacional, como listar diretórios, criar e remover arquivos, etc.

**2. Definição da Função `processar_respostas_alunos`:**

A função `processar_respostas_alunos` é definida com um parâmetro obrigatório `pasta_respostas`. Esse parâmetro representa o caminho para a pasta que contém as respostas dos alunos.

**3. Carregamento dos Conteúdos dos Arquivos:**

Dentro da função, três arquivos são abertos e seus conteúdos lidos em variáveis:

* `prompt.txt`: O conteúdo do arquivo `prompt.txt` é lido na variável `prompt`.
* `gabarito.txt`: O conteúdo do arquivo `gabarito.txt` é lido na variável `gabarito`.
* `resposta_aluno.txt`: Para cada arquivo na pasta de respostas, o conteúdo é lido na variável `resposta_aluno`.

**4. Inicialização do Dicionário de Respostas Processadas:**

Um dicionário vazio chamado `respostas_processadas` é criado para armazenar as respostas concatenadas.

**5. Iteração pelos Arquivos na Pasta:**

A função utiliza um loop `for` para iterar pelos arquivos na pasta `pasta_respostas`. Para cada arquivo:

* **Verificação da Extensão do Arquivo:** A função verifica se a extensão do arquivo termina com `.txt`. Se sim, prossegue para o próximo passo.
* **Separação do Nome do Arquivo e Extensão:** A função utiliza `os.path.splitext` para separar o nome do arquivo (sem extensão) e a extensão em variáveis `nome_arquivo` e `extensao`, respectivamente.
* **Abertura do Arquivo de Resposta do Aluno:** A função utiliza `os.path.join` para concatenar o caminho da pasta e o nome do arquivo, criando o caminho completo do arquivo de resposta do aluno. Esse arquivo é aberto em modo de leitura (`'r'`) utilizando a instrução `with open`.
* **Leitura do Conteúdo da Resposta do Aluno:** O conteúdo do arquivo de resposta do aluno é lido na variável `resposta_aluno`.
* **Concatenação dos Conteúdos:** A função `concatenar_strings` (que deve ser definida separadamente) é chamada para concatenar o prompt, a resposta do aluno e o gabarito em uma única string.
* **Armazenamento da Resposta Concatenada no Dicionário:** A string concatenada é armazenada no dicionário `respostas_processadas` utilizando o nome do arquivo (sem extensão) como chave.

**6. Retorno do Dicionário de Respostas:**

A função retorna o dicionário `respostas_processadas` contendo as respostas concatenadas para cada arquivo de resposta.


In [10]:
import os

def processar_respostas_alunos(pasta_respostas):
  """
  Função para processar todas as respostas dos alunos em uma pasta.

  Argumentos:
    pasta_respostas: O caminho para a pasta que contém as respostas dos alunos.

  Retorno:
    Um dicionário onde as chaves são os nomes dos arquivos de resposta e os valores são as strings concatenadas contendo as respostas, prompts e gabaritos.
  """

  # Carregar o conteúdo de prompt.txt, resposta_aluno.txt e gabarito.txt
  with open("prompt.txt") as prompt_file:
    prompt = prompt_file.read()

  with open("gabarito.txt") as gabarito_file:
    gabarito = gabarito_file.read()

  respostas_processadas = {}

  for arquivo in os.listdir(pasta_respostas):
    if arquivo.endswith(".txt"):
      nome_arquivo, extensao = os.path.splitext(arquivo)

      with open(os.path.join(pasta_respostas, arquivo), 'r') as resposta_aluno_file:
        resposta_aluno = resposta_aluno_file.read()

      # Concatenar os conteúdos usando a função concatenar_arquivos
      texto_concatenado = concatenar_strings(prompt, resposta_aluno, gabarito)

      # Armazenar o texto concatenado no dicionário com o nome do arquivo como chave
      respostas_processadas[nome_arquivo] = texto_concatenado

  return respostas_processadas

pasta_respostas = "respostas"
prompts_preparados =  processar_respostas_alunos(pasta_respostas)


##Escrever o feedback de cada resposta da IA no correspondente arquivo na pasta feedback.

## Instalação do Pacote `google-generativeai`

A instrução `!pip install -q -U google-generativeai` que você forneceu tem como objetivo instalar o pacote `google-generativeai` do Google Colab. Este pacote oferece acesso a modelos de linguagem e APIs de geração de texto avançados do Google AI.

**Detalhes da Instalação:**

* **`pip`:** O comando utiliza a ferramenta `pip` (Python Package Indexer) para gerenciar a instalação de pacotes Python.
* **`install`:** Especifica a ação de instalação de um pacote.
* **`-q`:** Avisa ao `pip` para suprimir a saída padrão, resultando em uma instalação silenciosa.
* **`-U`:** Indica que a instalação deve atualizar o pacote para a versão mais recente disponível.
* **`google-generativeai`:** Nome do pacote a ser instalado.


In [3]:
!pip install -q -U google-generativeai

## Configuração do SDK Python para o Google Generative AI

O código que você forneceu demonstra a configuração do SDK Python para o Google Generative AI. Vamos analisar cada linha para entender sua funcionalidade:

**1. Importação do Módulo:**

* `# Import the Python SDK` - (Comentário) Indica que o trecho a seguir é um comentário e não será executado pelo Python.
* `import google.generativeai as genai` - Importa o módulo `google.generativeai` e atribui o alias `genai` para facilitar a referência posterior. Este módulo fornece acesso às APIs do Google Generative AI para geração de texto, tradução e outras funcionalidades.

**2. Importação do Módulo `userdata`:**

* `from google.colab import userdata` - Importa o módulo `userdata` do Google Colab. Este módulo fornece ferramentas para acessar e gerenciar dados privados associados ao seu notebook do Colab.

**3. Recuperação da API Key:**

* `api_key = userdata.get('SECRET_KEY')` - Recupera o valor da chave secreta `SECRET_KEY` armazenada no armazenamento seguro do Colab utilizando o módulo `userdata.get`. A API Key é uma credencial necessária para autenticar e autorizar o uso do Google Generative AI.

**4. Configuração do SDK:**

* `genai.configure(api_key = api_key)` - Configura o SDK do Google Generative AI utilizando a API Key recuperada anteriormente. Essa configuração permite que o SDK use a chave correta para autenticação nas chamadas de API.

**Observações:**

* É crucial armazenar a API Key com segurança. O Google Colab oferece armazenamento seguro de dados privados através do módulo `userdata`.
* Substitua `'SECRET_KEY'` pelo nome real da chave secreta que você obteve do Google AI Platform.
* Certifique-se de ter uma conexão ativa com a internet para que a configuração do SDK seja bem-sucedida.

**Após a Configuração:**

Uma vez configurado, você pode utilizar o módulo `genai` (que é o alias para `google.generativeai`) para acessar os modelos e funcionalidades do Google Generative AI.


In [4]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key = api_key)

O dicionário `generation_config` que você forneceu define parâmetros para controlar a geração de texto com o Google Generative AI. Vamos analisar cada chave e valor para entender seus efeitos:

**1. `candidate_count`:**

* **Tipo:** Inteiro
* **Descrição:** Define o número de candidatos de texto a serem gerados pelo modelo. Um valor maior pode resultar em mais opções, mas também aumenta o tempo de processamento.
* **Valor padrão:** 1 (gera apenas um candidato)
* **Exemplos:**
    * `candidate_count = 2`: Gera dois candidatos de texto para cada prompt.
    * `candidate_count = 5`: Gera cinco candidatos de texto para cada prompt.

**2. `temperature`:**

* **Tipo:** Número real
* **Descrição:** Controla o nível de criatividade na geração de texto. Valores mais altos aumentam a probabilidade de gerar texto inesperado ou surpreendente, enquanto valores mais baixos favorecem a geração de texto mais previsível e semelhante ao texto de treinamento.
* **Valor padrão:** 0.9
* **Exemplos:**
    * `temperature = 0.5`: Gera texto mais conservador e próximo ao texto de treinamento.
    * `temperature = 1.5`: Gera texto mais criativo e original, com maior probabilidade de conter elementos inesperados.

**3. `top_p` (opcional):**

* **Tipo:** Número real
* **Descrição:** Define a probabilidade cumulativa máxima para cada token durante a geração de texto. Um valor mais alto aumenta a probabilidade de gerar tokens mais comuns no idioma, enquanto um valor mais baixo aumenta a probabilidade de gerar tokens raros ou inesperados.
* **Valor padrão:** Não definido (se não for especificado, o modelo usará um valor otimizado internamente)
* **Exemplos:**
    * `top_p = 0.9`: Prioriza tokens mais comuns, resultando em texto mais previsível.
    * `top_p = 0.1`: Permite que tokens raros ou inesperados sejam gerados com mais frequência, resultando em texto mais criativo e original.

**Observações:**

* A escolha dos valores para `candidate_count`, `temperature` e `top_p` depende da sua necessidade específica e do tipo de texto que você deseja gerar. Experimente diferentes valores para encontrar a combinação que melhor se adapta aos seus objetivos.
* Esses parâmetros podem afetar a qualidade e o tempo de processamento da geração de texto.
* Consulte a documentação oficial do Google Generative AI para obter mais informações sobre os parâmetros de geração e como ajustá-los para obter os melhores resultados.


In [5]:
generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.9, # Nível de criatividade
    #"top_p" : 0.5,
}

## Configuração de Filtros de Segurança para o Google Generative AI

O dicionário `safety_settings` que você forneceu define as configurações para os filtros de segurança do Google Generative AI. Esses filtros ajudam a mitigar a geração de texto prejudicial ou inadequado. Vamos analisar cada chave e valor para entender sua funcionalidade:

**Chaves:**

* **"HARASSMENT"**: Determina o nível de bloqueio para conteúdo que possa ser considerado assédio.
* **"HATE"**: Determina o nível de bloqueio para conteúdo que possa ser considerado discurso de ódio.
* **"SEXUAL"**: Determina o nível de bloqueio para conteúdo sexualmente explícito.
* **"DANGEROUS"**: Determina o nível de bloqueio para conteúdo perigoso ou violento.

**Valores:**

* **"BLOCK_NONE"**: Desabilita o filtro de segurança para a respectiva categoria. O modelo poderá gerar conteúdo que se enquadre nessa categoria.
* **"BLOCK_SOME"**: Ativa o filtro de segurança com bloqueio moderado. O modelo tentará evitar gerar conteúdo prejudicial, mas pode haver vazamentos ocasionais.
* **"BLOCK_MOST"**: Ativa o filtro de segurança com bloqueio rigoroso. O modelo tentará evitar gerar a maioria do conteúdo prejudicial, mas é possível que alguns casos escapem.


In [6]:
safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE",
}

## Criação de um Modelo Generativo no Google Generative AI

O código que você forneceu demonstra a criação de um modelo generativo utilizando a biblioteca `google.generativeai`. Vamos analisar cada parte para entender sua função:

**1. Importação do Módulo (já pressupomos que ele foi importado):**

* `from genai.models import GenerativeModel` (assumindo que a importação já foi realizada anteriormente).

**2. Criação do Modelo:**

* `model = genai.GenerativeModel(model_name= "gemini-1.0-pro", generation_config = generation_config, safety_settings = safety_settings)` - Esta linha cria uma instância da classe `GenerativeModel`.

**Parâmetros:**

* `model_name` (Obrigatório): Nome do modelo generativo que você deseja utilizar. No caso, `"gemini-1.0-pro"`. Consulte a documentação do Google Generative AI para obter uma lista de modelos disponíveis.
* `generation_config` (opcional): Dicionário contendo parâmetros para controlar a geração de texto (já documentado anteriormente).
* `safety_settings` (opcional): Dicionário contendo configurações para os filtros de segurança (já documentado anteriormente).

**Observações:**

* É necessário ter uma conexão ativa com a internet para criar o modelo.
* A autenticação com a API Key configurada anteriormente (através de `genai.configure`) é necessária para acessar o modelo.
* Você pode criar vários modelos com diferentes configurações de geração e segurança utilizando a mesma classe `GenerativeModel`.


In [7]:
model = genai.GenerativeModel(model_name= "gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

## Geração e Armazenamento de Feedback com Google Generative AI

O código a seguir demonstra como gerar e armazenar feedback de prompts utilizando o Google Generative AI.

O código cria arquivos de texto individuais para cada prompt na pasta feedback.

Você pode modificar o formato do nome do arquivo ou a estrutura de armazenamento do feedback de acordo com suas necessidades.



In [11]:
import os

# Crie a pasta feedback se ela não existir
if not os.path.exists('feedback'):
    os.makedirs('feedback')

# Dicionário com as strings de feedback
feedback = {}

for prompt in prompts_preparados:
  response = model.generate_content(prompts_preparados[prompt])
  feedback[prompt] = response.text

# Percorra o dicionário
for key, value in feedback.items():
    # Crie um nome de arquivo usando a chave
    filename = f'feedback/{key}.txt'

    # Abra o arquivo em modo de escrita
    with open(filename, 'w') as f:
        # Escreva o feedback no arquivo
        f.write(value)

##Ideia de aprimoramento deste projeto

Aprimorar este Script Python para analisar códigos Python coletados através de um formulário Google, comparando o código Python do aluno com um código gabarito de cada questão localizado em uma planilha Google. As respostas e orientações obtidas com a IA devem são armazenadas em uma planilha.